In [27]:
import pandas as pd
from collections import Counter
from PIL import Image,ImageDraw,ImageFont
from scipy.stats import spearmanr
import os
import numpy as np

In [28]:
highlight = True

In [29]:
limefeat_alphasort = pd.read_csv("limefeat_alphasort.csv")

#### Color settings

In [35]:
cathue = {
    'ts':'0',
    'fi':'32',
    'ir':'60',
    'shifts':'115',
    'dispatch':'180',
    'arrests':'240',
    'ic':'270',
    'demarrests':'300',
    'ocnd':'200'
    }

bkgdcolor = 'hsl(0,0%,50%)'
linecolordark = 'hsl(0,0%,0%)'
linecolormed = 'hsl(0,0%,25%)'
linecolorlight = 'hsl(0,0%,33%)'
textcolor = 'hsl(0,0%,0%)'

#### X

In [42]:
X = pd.read_csv("32092_2014-04-11_test-matrix-and-labels.csv")
X_pred = pd.read_csv("32092_2014-04-11_predictions.csv")
X_pred.rename(columns={"entity_id":"officer_id"},inplace=True)

In [44]:
del X['as_of_date']
tmp = X_pred[['officer_id','score']]
tmp.set_index('officer_id',inplace=True)
X = X.join(tmp,on='officer_id')
X.sort_values(by='score',ascending=False,inplace=True)

In [ ]:
X.reset_index(drop=True,inplace=True)
del X['outcome']
del X['officer_id']
del X['score']

#### Mapping saturation to data values

In [45]:
from scipy.stats import percentileofscore as pct

In [46]:
Xpct = X

In [47]:
def norm(col):
    rng = max(col) - min(col)
    tmp = [(item - min(col)) / float(rng) for item in col]
    return [int(round(item*100)) for item in tmp]

In [48]:
counter=-1
for col in X.columns:
    counter+=1
    if np.random.binomial(1,0.01)==1:
        print counter
    if all(X[col]<=0):
        Xpct[col] = 0
    elif 'ocnd' in col:
        for i in X.index:
            if X[col].loc[i]==1:
                Xpct[col].loc[i] = 65
            elif X[col].loc[i]==0:
                Xpct[col].loc[i] = 25
    else:
        Xpct[col] = norm(X[col])

3
140
156
199
362
394
411
630
659
676
722
754
843
889
1388
1394
1430
1458


In [49]:
rectwidth = 10
rectheight = 10
px_w = len(limefeat_alphasort) * rectwidth
px_h = len(X) * rectheight

In [65]:
def offtablefeat(X,feature_table):
    canvas = Image.new('RGB',(px_w,px_h),bkgdcolor)
    feats = X.columns
    for i in X.index:
        ycoord = i * rectheight
        
        for feat in feats:
            tmp = feature_table[feature_table.feature==feat]
            record_type = tmp.record_type.iloc[0]
            idx = tmp.index[0]
            
            xcoord = idx * rectwidth
            bbox = [xcoord, ycoord, xcoord + rectwidth, ycoord + rectheight]
            
            try:
                hue = cathue[record_type]
                satlevel = Xpct[feat].loc[i]
                sat = str(satlevel)+'%'
                
                if highlight==True:
                    if X_pred.label_value[X_pred.officer_id==i].iloc[0]==1:
                        val = '65%'
                    else:
                        val = '25%'
                else:
                    val = '50%'
                    
            except: # white
                hue = '0'
                sat = '0%'
                val = '100%'

            fillcolor = 'hsl('+hue+","+sat+","+val+")"
            draw = ImageDraw.Draw(canvas)
            draw.rectangle(bbox,fill=fillcolor,outline=None)
    return canvas

In [66]:
im = offtablefeat(X,limefeat_alphasort)
w = im.width
h = im.height
x = 50
canvas = Image.new('RGB',((w+x),(h)),bkgdcolor)
canvas.paste(im,(x,0))
im = None

In [67]:
draw = ImageDraw.Draw(canvas)
for i in X_pred.index:
    officer_id = X_pred.officer_id.loc[i]
    text = str(officer_id)
    font = ImageFont.truetype('VeraMono.ttf', rectheight)
    fontWidth, fontHeight = font.getsize(text)
    
    xpos = 5
    ypos = i * rectheight
    draw.text((xpos,ypos),text=text,font=font,fill=textcolor)
    
    linex = w+x
    liney = i * rectheight
    draw.line([(0,liney),(linex,liney)],width=1,fill=linecolordark)

In [68]:
w = canvas.width
h = canvas.height
y = 400
unicanvas = Image.new('RGB',((w),(h+y)),bkgdcolor)
unicanvas.paste(canvas,(0,y))
canvas = None

In [69]:
draw = ImageDraw.Draw(unicanvas)
for i in limefeat_alphasort.index:
    feature = limefeat_alphasort.featstr.loc[i]
    text = str(feature)
    
    font = ImageFont.truetype('VeraMono.ttf', rectwidth)
    fontWidth, fontHeight = font.getsize(text)
    
    imtmp = Image.new('RGBA', (fontWidth, fontHeight))
    drawtmp = ImageDraw.Draw(imtmp)
    drawtmp.text((0,0),text=text,font=font,fill=textcolor)
    imtmp = imtmp.rotate(90,expand=1)
    
    xpos = i * rectwidth + x - 3
    ypos = y - imtmp.height - 2
    unicanvas.paste(imtmp,(xpos,ypos),imtmp)
    
    linex = i * rectwidth + x
    liney = y
    draw.line([(linex,liney),(linex,unicanvas.height)],width=1,fill=linecolordark)

In [70]:
unicanvas.save("/Users/damoncrockett/Desktop/X_"+str(highlight)+".png")